In [2]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

# Initialize database

In [3]:
from pymongo import MongoClient

client = MongoClient()

db = client['glvis_db']

# Tranfrom Glove format to word2vec format

In [41]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [46]:
glove_input_file = 'data/glove.6B/glove.6B.300d.txt'
word2vec_ouput_file = glove_input_file + '.word2vec'

In [47]:
glove2word2vec(glove_input_file, word2vec_ouput_file)

(400000, 300)

# Save word2vec to database

In [6]:
model = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [8]:
for w in model.vocab:
    entry = {
        'word': w,
        'embedding': model.get_vector(w).tolist()
    }
    
    db['word2vec'].insert_one(entry)

# Save Glove embeddings to database

In [6]:
model = KeyedVectors.load_word2vec_format('data/glove.6B/glove.6B.300d.txt.word2vec', binary=False)

In [7]:
entries = []
for w in model.vocab:
    entry = {
        'word': w,
        'embedding': model.get_vector(w).tolist()
    }
    
    entries.append(entry)

In [8]:
db['glove_6b_300d'].insert_many(entries)

# Experiement with filtering words

In [6]:
s = 'quickly'

In [13]:
ly_adv = list(filter(lambda x: x.endswith('ly'), vocab))

In [17]:
import random

In [22]:
random.sample(ly_adv, k=1000)

['funnily',
 'ecstatically',
 'diametrically',
 'dourly',
 'deferentially',
 'gushingly',
 'atypically',
 'preciously',
 'exceedingly',
 'supremely',
 'academically',
 'damselfly',
 'frenetically',
 'anxiously',
 'musicianly',
 'joyously',
 'vocally',
 'helmly',
 'companionably',
 'synthetically',
 'gingerly',
 'craftily',
 'searingly',
 'squarely',
 'absurdly',
 'unsparingly',
 'concentrically',
 'disconsolately',
 'uncomplainingly',
 'exactingly',
 'gracefully',
 'saly',
 'inescapably',
 'harmoniously',
 'quarterly',
 'artistically',
 'contemporaneously',
 'absently',
 'keneally',
 'enjoyably',
 'barbarically',
 'nearly',
 'posteriorly',
 'willy',
 'rully',
 'effectually',
 'neily',
 'neurally',
 'electronically',
 'obviously',
 'sluggishly',
 'botfly',
 'insultingly',
 'jelly',
 'demonstrably',
 'unbearably',
 'curmudgeonly',
 'seasonably',
 'wearily',
 'profanely',
 'unselfishly',
 "o'kelly",
 'faintly',
 'wizardly',
 'inseparably',
 'luridly',
 'unsustainably',
 'gently',
 'euphor

In [24]:
import nltk

In [32]:
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/snie/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /home/snie/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [29]:
pos_tags = nltk.pos_tag(vocab)

In [36]:
RB = list(map(lambda x: x[0], filter(lambda x: x[1] == 'RB', pos_tags)))

In [39]:
len(RB)

7013

In [ ]:
nltk.help.upenn_tagset()

In [7]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431607246399),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

In [31]:
vocab = np.array(list(model.vocab.keys()))

In [23]:
import random

In [51]:
from sklearn.manifold import TSNE

In [56]:
embedded = TSNE(n_components=2).fit_transform(vectors)

In [63]:
vocab = list(model.vocab.keys())

sample_words = [vocab[i] for i in random.sample(range(0, len(vocab)), sample_size)]

sample_vectors = [model.get_vector(w) for w in sample_words]

embedded_vectors = TSNE(n_components=2).fit_transform(np.array(vectors))